In [1]:
from azure.ai.ml import MLClient

In [2]:
pip uninstall azure-ai-ml --yes

Found existing installation: azure-ai-ml 0.1.0b6
Uninstalling azure-ai-ml-0.1.0b6:
  Successfully uninstalled azure-ai-ml-0.1.0b6
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install --pre azure-ai-ml

  Using cached azure_ai_ml-0.1.0b6-py3-none-any.whl (2.7 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install mlflow argparse pandas lightgbm matplotlib sklearn

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=a3562ce1e232556f8eb827c6162a6cbaa061d516175aa6e6dde5c73ca55fe252
  Stored in directory: /home/azureuser/.cache/pip/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn
ERROR: pyldavis 3.3.1 has requirement numpy>=1.20.0, but you'll have numpy 1.19.0 which is incompatible.
ERROR: pyldavis 3.3.1 has requirement pandas>=1.2.0, but you'll have pandas 1.1.5 which is incompatible.
ERROR: pycaret 2.3.10 has requirement pyyaml<6.0.0, but you'll have pyyaml 6.0 which is incompatible.
ERROR: pycaret 2.3.10 has requirement scikit-learn==0.23.2, but you'll have scikit-learn 0.22.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Import required libraries

# Credentials
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
# Machine Learning Workspace
from azure.ai.ml import MLClient
# Compute Target
from azure.ai.ml.entities import AmlCompute
# Command and Arguments
from azure.ai.ml import command, Input

In [2]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    # This will open a browser page for
    credential = InteractiveBrowserCredential()

In [3]:
# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="630fbb67-7809-49f9-af9c-55704307b953",
    resource_group_name="RG_MachineLearning",
    workspace_name="mlaiws"
)
print(ml_client)

MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7fd583b3b9a0>,
         subscription_id=630fbb67-7809-49f9-af9c-55704307b953,
         resource_group_name=RG_MachineLearning,
         workspace_name=mlaiws)


In [4]:
compute_target_name = "ComputeClusterDemo"

try:
    # Check if the compute target already exists
    compute_target = ml_client.compute.get(compute_target_name)
    print(f"{compute_target.name} of node size {compute_target.size} is reused.")
except Exception:
    compute_target = AmlCompute(
        name=compute_target_name,
        type="amlcompute",
        size="Standard_DS3_v2",
        min_instances=0,
        max_instances=1,
        idle_time_before_scale_down=120,
        tier="Dedicated"
    )
    compute_target = ml_client.begin_create_or_update(compute_target)
    print(f"{compute_target.name} of node size {compute_target.size} is created.")

ComputeClusterDemo of node size STANDARD_DS3_V2 is reused.


In [5]:
# Define the command
command_job = command(
    code="./src",
    command="python main.py --iris-csv ${{inputs.csv}} --learning-rate ${{inputs.rate}} --boosting ${{inputs.boost}}",
    environment="AzureML-lightgbm-3.2-ubuntu18.04-py37-cpu@latest",
    inputs={
        "csv": Input(
            type="uri_file",
            path="https://azuremlexamples.blob.core.windows.net/datasets/iris.csv",
        ),
        "rate": 0.9,
        "boost": "gbdt",
    },
    compute="ComputeClusterDemo",
)

In [6]:
# Submit the command
returned_job = ml_client.jobs.create_or_update(command_job)
# Get a URL for the status of the job
print(returned_job.services["Studio"].endpoint)

Uploading src (0.0 MBs): 100%|██████████| 2825/2825 [00:00<00:00, 194330.42it/s]




https://ml.azure.com/runs/jolly_pear_qlcwymdn6f?wsid=/subscriptions/630fbb67-7809-49f9-af9c-55704307b953/resourcegroups/RG_MachineLearning/workspaces/mlaiws&tid=d18376e2-1716-4c44-bb34-672b431bebc2
